### Imports

In [92]:
import requests
import json
from bs4 import BeautifulSoup

2. Sending HTTP request to acces first page with options

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews_scroll"

response = requests.request(method="GET" , url=url)
response.status_code

3. Checking the code of Http responce

In [ ]:
response.status_code

4. Parse the HTML code of first page with options

In [ ]:
page_dom= BeautifulSoup(response.text, "html.parser")
print(page_dom.prettify())

5. Extract required data from parsed code

In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
opinion = page_dom.select_one("div.js_product-review:not(.user-post--highlight)")
print(type(page_dom))
print(type(opinion))
print(opinion.prettify())

In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")

for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recommendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
    except AttributeError:
        recommendation = None

    stars= opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
    cons = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
    vote_yes = opinion.select_one("button.vote-yes[data-total-vote]")
    vote_no = opinion.select_one("button.vote-no[data-total-vote]")
    published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
    try:
        purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    except TypeError:
        purched = None
    print(opinion_id, author, recommendation, stars, content, pros, cons, vote_yes, vote_no, published, published, sep="\n")

6. if there is no pages move to next page

In [ ]:
try:
    next_page="https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
except TypeError:
    next_page=None


In [ ]:
next_page="https://www.ceneo.pl/84514582#tab=reviews"
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        for opinion in opinions:
            single_opinion={}
            single_opinion["opinion_id"]=opinion["data-entry-id"]
            single_opinion["author"]=opinion.select_one("span.user-post__author-name").get_text()
            author = opinion.select_one("span.user-post__author-name").get_text()
            try:
                recommendation = opinion.select_one("span.user-post__author-recomendation > em").get_text()
            except AttributeError:
                recommendation = None

            stars= opinion.select_one("span.user-post__score-count").get_text()
            content = opinion.select_one("div.user-post__text").get_text()
            pros = [p.get_text() for p in opinion.select("div.review-feature__item--positive")]
            cons = [c.get_text() for c in opinion.select("div.review-feature__item--negative")]
            vote_yes = opinion.select_one("button.vote-yes[data-total-vote]")
            vote_no = opinion.select_one("button.vote-no[data-total-vote]")
            published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
            try:
                purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
            except TypeError:
                purched = None
            print(single_opinion)    
        try:
            next_page = "https://www.ceneo.pl" + page_dom.select_one("a.pagination__next")["href"]
        except TypeError:
            next_page = None